# 많은 도구 관리: 동적 도구 선택

이 노트북에서는 **많은 도구 중에서 관련 도구만 선택**하는 에이전트를 만듭니다.

## 왜 동적 도구 선택이 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    많은 도구의 문제점                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   도구가 많아지면:                                                 │
│                                                                    │
│   1️⃣ 컨텍스트 윈도우 낭비                                         │
│      • 모든 도구 설명을 LLM에 전달해야 함                          │
│      • 토큰 비용 증가, 응답 속도 저하                              │
│                                                                    │
│   2️⃣ LLM 혼란                                                     │
│      • 비슷한 도구들 중 선택 어려움                                │
│      • 잘못된 도구 선택 가능성 증가                                │
│                                                                    │
│   해결책:                                                          │
│   질문과 관련된 도구만 선택 → 선택된 도구만 LLM에 전달             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    동적 도구 선택 아키텍처                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │ select_tools │  ← 도구 설명을 벡터 검색하여 관련 도구 선택     │
│   └──────┬───────┘                                                 │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │    model     │  ← 선택된 도구만 bind_tools                     │
│   └──────┬───────┘                                                 │
│          │                                                         │
│      (조건부)─────────────┐                                        │
│          │                │                                        │
│          ▼                ▼                                        │
│   ┌──────────────┐   ┌─────────┐                                   │
│   │    tools     │   │   END   │                                   │
│   └──────┬───────┘   └─────────┘                                   │
│          │                                                         │
│          └───────────────────────────────────────┐                 │
│                                                  │                 │
│                                              model                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 도구 정의

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

print("✅ 도구 정의 완료")
for t in tools:
    print(f"   - {t.name}: {t.description[:50]}...")

# 3. 도구 Retriever 생성

도구 설명을 벡터화하여 검색할 수 있게 합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구 Retriever 동작                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1. 각 도구의 설명(description)을 Document로 변환                 │
│   2. Document를 벡터로 임베딩하여 저장                             │
│   3. 사용자 질문으로 유사한 도구 설명 검색                         │
│   4. 검색된 도구만 LLM에 전달                                      │
│                                                                    │
│   예시:                                                            │
│   질문: "123 + 456은?"                                            │
│   → "계산기" 설명과 유사도 높음                                    │
│   → calculator 도구 선택                                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.documents import Document
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델
embeddings = OllamaEmbeddings(model='nomic-embed-text')

# 도구 설명을 Document로 변환
tool_documents = [
    Document(
        page_content=tool.description,  # 도구 설명
        metadata={'name': tool.name}     # 도구 이름
    )
    for tool in tools
]

print("=== 도구 Document ===")
for doc in tool_documents:
    print(f"  {doc.metadata['name']}: {doc.page_content}")

In [ ]:
# 벡터 스토어 생성 및 Retriever 변환
tools_retriever = InMemoryVectorStore.from_documents(
    tool_documents,
    embeddings
).as_retriever()

print("✅ 도구 Retriever 생성 완료")

In [ ]:
# Retriever 테스트
test_queries = [
    "100 + 200을 계산해줘",
    "오늘 뉴스 검색해줘",
]

print("=== Retriever 테스트 ===")
for query in test_queries:
    results = tools_retriever.invoke(query)
    print(f"\n질문: {query}")
    print(f"선택된 도구: {[doc.metadata['name'] for doc in results]}")

# 4. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

# 도구 없이 기본 모델 생성 (나중에 동적으로 bind_tools)
model = ChatOllama(model='llama3.2', temperature=0.1)

print("✅ LLM 설정 완료")
print("   도구는 select_tools 노드 후에 동적으로 연결됨")

# 5. State 정의 (확장)

선택된 도구 목록을 저장하기 위해 State를 확장합니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    확장된 State
    
    messages: 대화 기록
    selected_tools: 선택된 도구 이름 목록 (새로 추가!)
    """
    messages: Annotated[list, add_messages]
    selected_tools: list[str]  # 선택된 도구 이름

print("✅ State 정의 완료")
print("   - messages: 대화 기록")
print("   - selected_tools: 선택된 도구 목록")

# 6. 노드 정의

In [ ]:
def select_tools(state: State) -> State:
    """
    도구 선택 노드
    
    1. 사용자 질문 가져오기
    2. Retriever로 관련 도구 검색
    3. 선택된 도구 이름 반환
    """
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    
    return {
        'selected_tools': [doc.metadata['name'] for doc in tool_docs]
    }


def model_node(state: State) -> State:
    """
    LLM 호출 노드 (동적 도구 연결)
    
    1. selected_tools에 있는 도구만 필터링
    2. 필터링된 도구로 bind_tools
    3. LLM 호출
    """
    # 선택된 도구만 필터링
    selected_tools = [
        tool for tool in tools 
        if tool.name in state['selected_tools']
    ]
    
    # 동적으로 도구 연결 후 호출
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    
    return {'messages': res}


print("✅ 노드 정의 완료")
print("   - select_tools: 관련 도구 선택")
print("   - model_node: 선택된 도구만 사용하여 LLM 호출")

# 7. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

# 노드 추가
builder.add_node('select_tools', select_tools)  # 도구 선택 노드
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))

# 엣지 추가
builder.add_edge(START, 'select_tools')         # 시작 → 도구 선택
builder.add_edge('select_tools', 'model')       # 도구 선택 → 모델
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 8. 에이전트 실행

In [ ]:
from langchain_core.messages import HumanMessage

# 테스트
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 동적 도구 선택 에이전트 실행 ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if 'selected_tools' in node_output:
            print(f"  선택된 도구: {node_output['selected_tools']}")
        print()

In [ ]:
# 최종 결과
result = graph.invoke(input_data)

print("=== 최종 결과 ===")
print(f"선택된 도구: {result.get('selected_tools', [])}")
print(f"\n응답: {result['messages'][-1].content}")

# 9. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "123 * 456은 얼마인가요?",              # 계산기
    "오늘 한국의 날씨는 어떤가요?",         # 검색
    "파이썬에서 리스트를 정렬하는 방법",    # 검색
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print("="*60)
    
    result = graph.invoke({'messages': [HumanMessage(question)]})
    
    print(f"\n선택된 도구: {result.get('selected_tools', [])}")
    print(f"\n응답: {result['messages'][-1].content[:200]}..." if len(result['messages'][-1].content) > 200 else f"\n응답: {result['messages'][-1].content}")

---

## 정리: 동적 도구 선택

### 아키텍처

```
┌─────────────────────────────────────────────────────────────────────┐
│                    동적 도구 선택 흐름                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   도구 설명 → 임베딩 → 벡터 DB 저장                                │
│                                                                     │
│   사용자 질문 → 벡터 검색 → 관련 도구 선택                         │
│                                                                     │
│   선택된 도구만 LLM에 전달 → 효율적인 처리                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 도구 Retriever 생성
tool_documents = [
    Document(tool.description, metadata={'name': tool.name})
    for tool in tools
]
tools_retriever = InMemoryVectorStore.from_documents(
    tool_documents, embeddings
).as_retriever()

# 2. 도구 선택 노드
def select_tools(state):
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}

# 3. 동적 bind_tools
def model_node(state):
    selected = [t for t in tools if t.name in state['selected_tools']]
    res = model.bind_tools(selected).invoke(state['messages'])
    return {'messages': res}
```

### 장점

| 장점 | 설명 |
|------|------|
| **토큰 절약** | 필요한 도구 설명만 전달 |
| **응답 속도** | 컨텍스트 줄어서 빨라짐 |
| **정확도** | LLM이 선택할 도구가 적어져 혼란 감소 |
| **확장성** | 도구가 수십 개여도 효율적 처리 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## ch06 요약: 에이전트 패턴

| 패턴 | 용도 | 핵심 |
|------|------|------|
| **기본 ReAct** | LLM이 도구 사용 판단 | tools_condition |
| **강제 호출** | 항상 특정 도구 먼저 | ToolCall 직접 생성 |
| **동적 선택** | 많은 도구 효율화 | 도구 Retriever |

## 다음 단계

**ch07에서는** Reflection, Subgraph, Supervisor 등 **고급 에이전트 패턴**을 배웁니다.